How is the pref score changing with noisy data (or smoother data)

In [ ]:
import math
import pandas as pd
import altair as alt
import numpy as np
import flammkuchen as fl

import sys,os,os.path
sys.path.append(os.path.expanduser('../src'))

from spinorama.compute_cea2034 import estimated_inroom
from spinorama.compute_scores import scores
from spinorama.graph import graph_params_default, graph_freq, graph_spinorama
from spinorama.load import filter_graphs, load_normalize

df_all = fl.load('../cache.parse_all_speakers.h5')

Test 2 strategies: add noise and smooth by average over a rolling window

In [ ]:
def noise(df, amp):
    sdf = df.copy()
    l = len(sdf.Freq.values)
    for c in sdf.columns:
        if c != 'Freq':
            sdf[c] += amp*(np.random.rand(l)-0.5)
    return sdf

def smooth(df, amp):
    sdf = df.copy()
    l = len(sdf.Freq.values)
    for c in sdf.columns:
        if c != 'Freq':
            sdf[c] = sdf[c].rolling(amp).mean()
    sdf.fillna(df)
    return sdf

In [ ]:
def eval_score_noise(df_all, speaker, amplitude):
    df_klippel = df_all[speaker]['ASR']['asr']
    spl_H = noise(df_klippel['SPL Horizontal_unmelted'], amplitude)
    spl_V = noise(df_klippel['SPL Vertical_unmelted'], amplitude)
    df_smoothed = load_normalize(filter_graphs(speaker, spl_H, spl_V))
    return scores(df_smoothed)['pref_score']

def eval_score_smooth(df_all, speaker, amplitude):
    df_klippel = df_all[speaker]['ASR']['asr']
    if amplitude == 0:
        spl_H = df_klippel['SPL Horizontal_unmelted']
        spl_V = df_klippel['SPL Vertical_unmelted']
    else:
        spl_H = smooth(df_klippel['SPL Horizontal_unmelted'], amplitude)
        spl_V = smooth(df_klippel['SPL Vertical_unmelted'], amplitude)
    df_smoothed = load_normalize(filter_graphs(speaker, spl_H, spl_V))
    return scores(df_smoothed)['pref_score']

In [ ]:
# speaker = 'Adam S2V'
speaker = 'Emotiva Airmotiv B1+'

In [ ]:
range_noise = np.linspace(0, 3, 12)
res_noise = [(a, eval_score_noise(df_all, speaker, a)) for a in range_noise]


In [ ]:
range_smooth = [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 20, 30]
res_smooth = [(a, eval_score_smooth(df_all, speaker, a)) for a in range_smooth]

In [ ]:
for i, (a, n) in enumerate(res_noise):
    print('Noise +/-{:.1f}dB score {:.1f}'.format(a, n))

In [ ]:
for i, (a, n) in enumerate(res_smooth):
    print('Size of rolling window #{:2d} score {:.1f}'.format(a, n))

In [ ]:
chart_noise = alt.Chart(pd.DataFrame({'Amplitude': range_noise, 'Score': [j for i,j in res_noise]})
).mark_line(
).encode(
    x=alt.X('Amplitude:N', axis=alt.Axis(format='.1f', title='Noise amplitude (dB)')), 
    y=alt.Y('Score:Q', scale=alt.Scale(domain=[4,7])),
).properties(width=400, title='Score v.s. Noise')
chart_smooth = alt.Chart(pd.DataFrame({'Window': range_smooth, 'Score': [j for i,j in res_smooth]})
).mark_line(
).encode(
    x=alt.X('Window:N', axis=alt.Axis(title='Size of the rolling window')),
    y=alt.Y('Score:Q', scale=alt.Scale(domain=[4,7])),
).properties(width=400, title='Score v.s. Smoothing')
(chart_noise | chart_smooth).properties(title=speaker)

In [ ]:
speakers = ['Adam S2V', 'Emotiva Airmotiv B1+', 'Genelec 8030C']

In [ ]:
range_noise = np.linspace(0, 3, 12)
range_smooth = [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 20, 30]

In [ ]:
df_noise = pd.DataFrame({'Amplitude': range_noise})
df_smooth = pd.DataFrame({'Window': range_smooth})

for speaker in speakers:
    res_noise = [(a, eval_score_noise(df_all, speaker, a)) for a in range_noise]
    df_noise[speaker] = [j for i,j in res_noise]
    res_smooth = [(a, eval_score_smooth(df_all, speaker, a)) for a in range_smooth]
    df_smooth[speaker] = [j for i,j in res_smooth]
    

In [ ]:
chart_noise = alt.Chart(df_noise.melt('Amplitude', var_name='Speakers', value_name='Score')
).mark_line(
).encode(
    x=alt.X('Amplitude:N', axis=alt.Axis(format='.1f', title='Noise amplitude (dB)')), 
    y=alt.Y('Score:Q', scale=alt.Scale(domain=[4,7])),
    color=alt.Color('Speakers:N'),
).properties(width=400, title='Score v.s. Noise')
chart_smooth = alt.Chart(df_smooth.melt('Window', var_name='Speakers', value_name='Score')
).mark_line(
).encode(
    x=alt.X('Window:N', axis=alt.Axis(title='Size of the rolling window')),
    y=alt.Y('Score:Q', scale=alt.Scale(domain=[4,7])),
    color=alt.Color('Speakers:N'),
).properties(width=400, title='Score v.s. Smoothing')

In [ ]:
chart_noise|chart_smooth